## Agile Hardware Design
***
# Queue Design Case Study

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="width:20%;float:right" />

by Peter Hanping Chen, based on:

1. UC Berkley Bootcamp: configuration file: load-ivy.sc
- https://github.com/freechipsproject/chisel-bootcamp
2. Prof. Scott Beamer, sbeamer@ucsc.edu
- [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* Designing for reuse
* Designing a Queue
* Iteratively improving Queue design

## Loading The Chisel Library Into a Notebook

In [1]:
// interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/013_Queue/001_Code/source/load-ivy.sc
Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/013_Queue/001_Code/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/013_Queue/001_Code/Main.sc #2

path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/013_Queue/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Goals for Reuse

* Need to recognize _pattern_ of functionality

* Include necessary parameterization and generation to support users' needs

## Planning for Progressive Design

* Reduce the complexity/challenge of any one step
* _Close the loop_ as early as possible, then augment/extend/revise
* Look for opportunities to defer features/optimizations to later
* While developing, re-evaluate plan and revise as needed
* Consider
  * What is the simplest thing I can implement?
  * How can I test it? (both at start and as it evolves)
  * Plotting a roadmap to order development of features/optimizations

## Case Study: Designing a Queue

* **Goal:** A _Queue_ with `Decoupled` interfaces on both sides
  * Would like to parameterize queue depth and data types
  * Power/performance/area (PPA) goals

* **How to get started:** deferring features
  * Parameters (queue depth & data type)
  * Performance (correct but slow ok at first)

<img src="images/queue.svg" alt="queue high-level" style="width:70%;margin-left:auto;margin-right:auto"/>

## First Attempt at Queue

* _Simplification_: only single entry
* _Behavior_: can enqueue if full and dequeueing (`pipe` is true), but can't bypass if empty (`flow` is false)

<img src="images/single.svg" alt="single-entry queue" style="width:60%;margin-left:auto;margin-right:auto"/>

## V0 - First Attempt at Queue

In [3]:
class MyQueue(val numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
}

class MyQueueV0(bitWidth: Int) extends MyQueue(1, bitWidth) {
    val entry = Reg(UInt(bitWidth.W))
    val full = RegInit(false.B)
    io.enq.ready := !full || io.deq.fire
    io.deq.valid := full
    io.deq.bits := entry
    when (io.deq.fire) {
        full := false.B
    }
    when (io.enq.fire) {
        entry := io.enq.bits
        full := true.B
    }
}

defined class MyQueue
defined class MyQueueV0

## Testing Our Queue - Scala Model

In [4]:
class QueueModel(numEntries: Int, pipe: Boolean=true) {
    val mq = scala.collection.mutable.Queue[Int]()

    var deqReady = false  // set externally
    def deqValid() = mq.nonEmpty
    // be sure to call attemptDeq before attemptEnq within a cycle
    def attemptDeq() = if (deqReady && deqValid) Some(mq.dequeue()) else None
    
    def enqReady() = mq.size < numEntries-1 || 
                    (mq.size == numEntries-1 && !deqReady) ||
                    (mq.size == numEntries-1 && deqReady && pipe)
    def attemptEnq(elem: Int): Unit = if (enqReady()) mq += elem    // implies enqValid
}

defined class QueueModel

## Testing Our Queue - Harness + Simulation

In [5]:
def simCycle(qm: QueueModel, c: MyQueue, enqValid: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    val deqResult = qm.attemptDeq()
    if (deqResult.isDefined)
        c.io.deq.bits.expect(deqResult.get.U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqValid.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqValid)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

defined function simCycle

## Testing Our Queue - Simulation

In [6]:
test(new MyQueueV0(8)) { c =>
    val qm = new QueueModel(c.numEntries)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=true,  deqReady=true,  3)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1)
Queue(3)
Queue()
test MyQueueV0 Success: 0 tests passed in 7 cycles in 0.040295 seconds 173.72 Hz


## Assessing MyQueue `V0`

* Accomplished
    * Implements queueing behavior
    * Parameterized data width (still limited to `UInt`)
* Shortcommings
    * Only one entry (_next goal_ to fix)

## Parameterizing Number of Queue Entries

* First attempt at parameterizing number of entries: _shift register_

<img src="images/shift.svg" alt="queue via shift register" style="width:60%;margin-left:auto;margin-right:auto"/>

## V1 - Parameterizing Number of Queue Entries

In [7]:
class MyQueueV1(numEntries: Int, bitWidth: Int) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 0)
    // enqueue into index numEntries-1 (last) and dequeue from index 0 (head)
    val entries = Seq.fill(numEntries)(Reg(UInt(bitWidth.W)))
    val fullBits = Seq.fill(numEntries)(RegInit(false.B))
    val shiftDown = io.deq.fire || !fullBits.head
    io.enq.ready := !fullBits.last || shiftDown
    io.deq.valid := fullBits.head
    io.deq.bits := entries.head
    when (shiftDown) { // dequeue / shift
        for (i <- 0 until numEntries - 1) {
            entries(i) := entries(i+1)
            fullBits(i) := fullBits(i+1)
        }
        fullBits.last := false.B
    }
    when (io.enq.fire) { // enqueue
        entries.last := io.enq.bits
        fullBits.last := true.B
    }
//     when (shiftDown || io.enq.fire) {  // not quite right
//         entries.foldRight(io.enq.bits){(thisEntry, lastEntry) => thisEntry := lastEntry; thisEntry}
//         fullBits.foldRight(io.enq.fire){(thisEntry, lastEntry) => thisEntry := lastEntry; thisEntry}
//     }
}

defined class MyQueueV1

## Testing MyQueueV1

In [8]:
def simCycleV1(qm: QueueModel, c: MyQueue, enqValid: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    if (c.io.deq.valid.peek.litToBoolean && deqReady) {    // oddity to handle bubbles
        assert(qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().get.U)
    }
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqValid.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqValid)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueueV1(3,8)) { c =>
    val qm = new QueueModel(c.numEntries)
    simCycleV1(qm, c, enqValid=false, deqReady=false)
    simCycleV1(qm, c, enqValid=true,  deqReady=false, 1)
    simCycleV1(qm, c, enqValid=true,  deqReady=true,  2)
    simCycleV1(qm, c, enqValid=false, deqReady=true)
    simCycleV1(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(1, 2)
Queue(2)
test MyQueueV1 Success: 0 tests passed in 7 cycles in 0.011491 seconds 609.19 Hz


defined function simCycleV1

## Assessing MyQueue `V1`

* Accomplished
    * Implements queueing behavior
    * Parameterized data width & _number of entries_
* Shortcommings
    * Long latency when queue is empty (all elements go through all entries)
    * Not good at handling bubbles midway (might even be buggy)

## Squishing Bubbles in Queue

* Use a _Priority Encoder_ to squeeze out bubbles
  * Insert in first free slot

<img src="images/priority.svg" alt="priority encoder queue" style="width:60%;margin-left:auto;margin-right:auto"/>

## V2 - Using Priority Encoder for Insertion

In [9]:
class MyQueueV2(numEntries: Int, bitWidth: Int) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 0)
    // enqueue into lowest empty and dequeue from index 0 (head)
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val fullBits = RegInit(VecInit(Seq.fill(numEntries)(false.B)))
    val emptyBits = fullBits map { !_ }
    io.enq.ready := emptyBits reduce { _ || _ } // any empties?
    io.deq.valid := fullBits.head
    io.deq.bits := entries.head
    when (io.deq.fire) { // dequeue & shift up
        fullBits.last := false.B
        for (i <- 0 until numEntries - 1) {
            entries(i) := entries(i+1)
            fullBits(i) := fullBits(i+1)
        }
    }
    when (io.enq.fire) { // priority enqueue
        val currFreeIndex = PriorityEncoder(emptyBits)
        val writeIndex = Mux(io.deq.fire, currFreeIndex - 1.U, currFreeIndex)
        entries(writeIndex) := io.enq.bits
        fullBits(writeIndex) := true.B
    }
}

defined class MyQueueV2

## Testing MyQueueV2

In [10]:
test(new MyQueueV2(4, 8)) { c =>
    val qm = new QueueModel(c.numEntries, false)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=false, deqReady=true)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1)
Queue(1, 2)
Queue(2)
Queue()
test MyQueueV2 Success: 0 tests passed in 8 cycles in 0.015366 seconds 520.62 Hz


## Assessing MyQueue `V2`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy

* Shortcommings
  * _Performance:_ can't simultaneously enqueue/dequeue to a full queue
  * _Power Efficiency:_ lots of bits toggling
  * _Potential Critical Path:_ priority encoder logic depth

## Keeping Data in Place with a Circular Buffer

* _Circular buffer_ uses two pointers (indices) and fixed size storage to make a FIFO
  * Insert new data at _in_ (and increment _in_)
  * Pop from _out_ (and increment _out_)
  * Wrap pointers around when they get to end
* How to tell when empty vs full?
  * First try: _empty_ when pointers are equal, _full_ when in+1 == out

<img src="images/circular.svg" alt="circular buffer" style="width:60%;margin-left:auto;margin-right:auto"/>

## V3 - Keeping Data in Place with Circular Buffer

In [11]:
class MyQueueV3(numEntries: Int, bitWidth: Int) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W))) // Mem?
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val empty = enqIndex === deqIndex
    val full = (enqIndex +% 1.U) === deqIndex
    io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
    }
}

defined class MyQueueV3

## Testing MyQueueV3

In [12]:
test(new MyQueueV3(4, 8)) { c =>
    val qm = new QueueModel(c.numEntries-1, false)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=true,  deqReady=false, 3)
    simCycle(qm, c, enqValid=true,  deqReady=false, 4)
    simCycle(qm, c, enqValid=false, deqReady=true)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(1, 2, 3)
Queue(1, 2, 3)
Queue(2, 3)
Queue(3)
test MyQueueV3 Success: 0 tests passed in 9 cycles in 0.014243 seconds 631.89 Hz


## Assessing MyQueue `V3`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy
  * _Efficient:_ less bits toggling and shallower logic

* Shortcommings
  * _Capacity:_ loose one entry (to detect if full), and must be power of 2
  * _Performance:_ can't simultaneously enqueue/dequeue to a full queue

## Reclaiming Last Entry

* _Problem:_ with circular buffer (initially), had to keep last entry empty to differentiate a full queue from an empty queue
    * Otherwise, if `enqIndex === deqIndex`, is it full or empty?
* _Solution:_ add an extra bit of state (`maybeFull`) to capture this corner case
    * If indices are equal and `maybeFull` $\Rightarrow$ _full_
    * If indices are equal and `!maybeFull` $\Rightarrow$ _empty_
    * If indices are not equal $\Rightarrow$ not full or empty (has room)

## V4 - Adding State (`maybeFull`) Track Last Entry

In [13]:
class MyQueueV4(numEntries: Int, bitWidth: Int) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val maybeFull = RegInit(false.B)
    val empty = enqIndex === deqIndex && !maybeFull
    val full = enqIndex === deqIndex && maybeFull
    io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
        when (enqIndex =/= deqIndex) {
            maybeFull := false.B
        }
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
        when ((enqIndex +% 1.U) === deqIndex) {
            maybeFull := true.B
        }
    }
}

defined class MyQueueV4

## Testing MyQueueV4

In [14]:
test(new MyQueueV4(4, 8)) { c =>
    val qm = new QueueModel(c.numEntries, false)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=true,  deqReady=false, 3)
    simCycle(qm, c, enqValid=true,  deqReady=false, 4)
    simCycle(qm, c, enqValid=false, deqReady=true)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(1, 2, 3)
Queue(1, 2, 3, 4)
Queue(2, 3, 4)
Queue(3, 4)
test MyQueueV4 Success: 0 tests passed in 9 cycles in 0.014845 seconds 606.27 Hz


## Assessing MyQueue `V4`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries (_can now use all of them all_)
  * Latency based on occupancy
  * _Efficient:_ less bits shifting and shallower logic

* Shortcommings
  * _Capacity:_ must be power of 2
  * _Performance:_ can't simultaneously enqueue/dequeue to a full queue

## V5 - Simultaneous Enqueue/Dequeue When Full

In [15]:
class MyQueueV5(numEntries: Int, bitWidth: Int) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val maybeFull = RegInit(false.B)
    val empty = enqIndex === deqIndex && !maybeFull
    val full = enqIndex === deqIndex && maybeFull
    io.enq.ready := !full || io.deq.ready  // NOTE: io.enq.ready now attached to io.deq.ready
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
        when (enqIndex =/= deqIndex) {
            maybeFull := false.B
        }
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
        when ((enqIndex +% 1.U) === deqIndex) {
            maybeFull := true.B
        }
    }
}

defined class MyQueueV5

## Testing MyQueueV5

In [16]:
test(new MyQueueV5(2, 8)) { c =>
    val qm = new QueueModel(c.numEntries, true)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=true,  deqReady=true,  3)
    simCycle(qm, c, enqValid=true,  deqReady=true,  4)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(2, 3)
Queue(3, 4)
Queue(4)
test MyQueueV5 Success: 0 tests passed in 8 cycles in 0.014121 seconds 566.53 Hz


## Assessing MyQueue `V5`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy
  * _Efficient:_ less bits shifting and shallower logic
  * Can now enqueue/dequeue in same cycle

* Shortcommings
  * _Capacity:_ must be power of 2
  * _Possible combinational loop_ more likely with `io.enq.ready` now attached to `io.deq.ready`

## V6 - Tidying up Code

In [17]:
class MyQueueV6(numEntries: Int, bitWidth: Int, pipe: Boolean=true) extends MyQueue(numEntries, bitWidth) {
    require(numEntries > 1)
//     require(isPow2(numEntries))    // no longer needed
    val entries = Mem(numEntries, UInt(bitWidth.W))
    val enqIndex = Counter(numEntries)
    val deqIndex = Counter(numEntries)
    val maybeFull = RegInit(false.B)
    val indicesEqual = enqIndex.value === deqIndex.value
    val empty = indicesEqual && !maybeFull
    val full = indicesEqual && maybeFull
    if (pipe)
        io.enq.ready := !full || io.deq.ready
    else
        io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex.value)
    when (io.deq.fire =/= io.enq.fire) {
        maybeFull := io.enq.fire
    }
    when (io.deq.fire) {
        deqIndex.inc()
    }
    when (io.enq.fire) {
        entries(enqIndex.value) := io.enq.bits
        enqIndex.inc()
    }
}

defined class MyQueueV6

## Testing MyQueueV6

In [18]:
test(new MyQueueV6(3, 8)) { c =>
    val qm = new QueueModel(c.numEntries)
    simCycle(qm, c, enqValid=false, deqReady=false)
    simCycle(qm, c, enqValid=true,  deqReady=false, 1)
    simCycle(qm, c, enqValid=true,  deqReady=false, 2)
    simCycle(qm, c, enqValid=true,  deqReady=false, 3)
    simCycle(qm, c, enqValid=true,  deqReady=true,  4)
    simCycle(qm, c, enqValid=true,  deqReady=true,  5)
    simCycle(qm, c, enqValid=false, deqReady=true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(1, 2, 3)
Queue(2, 3, 4)
Queue(3, 4, 5)
Queue(4, 5)
test MyQueueV6 Success: 0 tests passed in 9 cycles in 0.016800 seconds 535.72 Hz


## Assessing MyQueue `V6`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy
  * _Efficient:_ less bits shifting and shallower logic
  * Can now enqueue/dequeue in same cycle (optionally) and support non-power of 2 capacities

* Shortcommings
  * Data type is `UInt` - What about arbitrary data?